In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir("/content/gdrive/MyDrive")
!ls

 2020fall-ml
'CBT test Diagram.drawio'
'CEN 5011 '
'Changhong Ma.pdf'
'Colab Notebooks'
'CPT_Changhong Ma.pdf'
 CV.pdf
 Data_by_Participant
'Getting started.pdf'
 mind_reading.py
'New Panther Virtual Check In Module CERTIFICATE OF COMPLETION-Quiz Passed.pdf'
 oa
 __pycache__
 testing.ipynb
'UCD-User Account Home Page.drawio'
'Untitled Diagram.drawio'
'Untitled spreadsheet.gsheet'


In [ ]:
import mind_reading as mr
import pandas as pd
import re

In [ ]:
%cd "/content/gdrive/MyDrive/Data_by_Participant"

/content/gdrive/MyDrive/Data_by_Participant


In [ ]:
directory = os.fsencode("/content/gdrive/MyDrive/Data_by_Participant")
os.listdir(directory)

[b'001',
 b'007',
 b'004',
 b'003',
 b'006',
 b'010',
 b'012',
 b'011',
 b'009',
 b'016',
 b'020',
 b'023',
 b'017',
 b'013',
 b'021',
 b'019',
 b'015',
 b'014',
 b'018',
 b'033',
 b'026',
 b'027',
 b'029',
 b'031',
 b'030',
 b'025',
 b'034',
 b'024',
 b'032',
 b'044',
 b'038',
 b'040',
 b'043',
 b'041',
 b'036',
 b'039',
 b'042',
 b'037',
 b'035',
 b'048',
 b'049',
 b'055',
 b'053',
 b'054',
 b'046',
 b'047',
 b'050',
 b'051',
 b'052',
 b'cha',
 b'059',
 b'061',
 b'063',
 b'060',
 b'058',
 b'057',
 b'056']

In [ ]:
orig_participants = os.listdir(directory)
participants = []

for participant in orig_participants:
  # decode byte, make sure use the string type
  participant = participant.decode('utf-8')
  participants.append(participant)

# remove the 'cha' folder
participants.remove('cha')
participants

['001',
 '007',
 '004',
 '003',
 '006',
 '010',
 '012',
 '011',
 '009',
 '016',
 '020',
 '023',
 '017',
 '013',
 '021',
 '019',
 '015',
 '014',
 '018',
 '033',
 '026',
 '027',
 '029',
 '031',
 '030',
 '025',
 '034',
 '024',
 '032',
 '044',
 '038',
 '040',
 '043',
 '041',
 '036',
 '039',
 '042',
 '037',
 '035',
 '048',
 '049',
 '055',
 '053',
 '054',
 '046',
 '047',
 '050',
 '051',
 '052',
 '059',
 '061',
 '063',
 '060',
 '058',
 '057',
 '056']

In [ ]:
df = pd.DataFrame(index = ['SVC', 'DTC', 'NB', 'NN'])

def acc_df(df, acc_column, participant):
    '''
    Add accuracy for every participant to the whole results
    Args: 
        df: the dataframe of all results we have had
        acc_column: the dataframe of result we want to add 
        participant: participant number
    returns: 
        all accuracy results
    '''

    data = pd.DataFrame({f"Participant {participant}": acc_column})
    df[f"Participant {participant}"] = data[f"Participant {participant}"].values
    return df

In [ ]:
for participant in participants:
    # iterate all the folders

    for file in os.listdir(participant):
    # iterate all files in every folder, find out the one end with 'Cong.csv' and 'Incong.csv' as input data

        if file.endswith('Cong.csv'): file1 = f"{participant}/{file}" 
        if file.endswith('Incong.csv'): file2 = f"{participant}/{file}"

    # load in cong and incong data for them
    df1 = mr.load_data(file1)
    df2 = mr.load_data(file2)

    # concatenate such data 
    data = mr.concatenate_data(df1, df2)

    # find trials to later separate
    trials_index = mr.find_trials(data)

    # separate trials
    trials = mr.separate_trials(data, trials_index)

    # create the label column  
    labels = mr.create_binary_labels(data)

    # Go through each trial, reset the columns, we split from 100-300ms ((308th sample to 513th sample))
    pro_trials = mr.process_trials(trials)

    # Find the mean across channels
    avg_trials = mr.average_trials(pro_trials)

    # concatenates the average trials dataframe with labels
    ml_df = mr.create_ml_df(avg_trials, labels)

    # train models
    X_train, X_test, y_train, y_test = mr.prepare_ml_df(ml_df)

    acc_svc, precision_svc = mr.train_svc(X_train, X_test, y_train, y_test)

    acc_dtc, precision_dtc = mr.train_dtc(X_train, X_test, y_train, y_test)

    acc_nb, precision_nb  = mr.train_nb(X_train, X_test, y_train, y_test)

    acc_nn, precision_nn = mr.train_nn(64, X_train, X_test, y_train, y_test)

    # add every participant's accuracy together
    acc_list = [f"{acc_svc:.2f}", f"{acc_dtc:.2f}", f"{acc_nb:.2f}", f"{acc_nn:.2f}"]

    df = acc_df(df, acc_list, participant)    

        

    

In [ ]:
df

,Participant 001,Participant 007,Participant 004,Participant 003,Participant 006,Participant 010,Participant 012,Participant 011,Participant 009,Participant 016,Participant 020,Participant 023,Participant 017,Participant 013,Participant 021,Participant 019,Participant 015,Participant 014,Participant 018,Participant 033,Participant 026,Participant 027,Participant 029,Participant 031,Participant 030,Participant 025,Participant 034,Participant 024,Participant 032,Participant 044,Participant 038,Participant 040,Participant 043,Participant 041,Participant 036,Participant 039,Participant 042,Participant 037,Participant 035,Participant 048,Participant 049,Participant 055,Participant 053,Participant 054,Participant 046,Participant 047,Participant 050,Participant 051,Participant 052,Participant 059,Participant 061,Participant 063,Participant 060,Participant 058,Participant 057,Participant 056
SVC,0.63,0.54,0.47,0.57,0.58,0.45,0.54,0.52,0.63,0.53,0.52,0.56,0.52,0.47,0.54,0.50,0.55,0.62,0.56,0.55,0.63,0.50,0.49,0.46,0.51,0.50,0.49,0.54,0.46,0.53,0.54,0.46,0.56,0.56,0.50,0.60,0.50,0.45,0.48,0.47,0.45,0.47,0.46,0.52,0.52,0.56,0.51,0.49,0.58,0.56,0.56,0.49,0.46,0.47,0.46,0.54
DTC,0.55,0.54,0.48,0.54,0.61,0.52,0.53,0.42,0.56,0.46,0.53,0.63,0.46,0.56,0.58,0.50,0.53,0.53,0.50,0.47,0.50,0.53,0.51,0.49,0.49,0.49,0.43,0.57,0.51,0.52,0.49,0.46,0.44,0.48,0.50,0.42,0.50,0.50,0.54,0.50,0.56,0.48,0.49,0.46,0.46,0.49,0.49,0.50,0.44,0.48,0.58,0.49,0.50,0.55,0.47,0.48
NB,0.47,0.54,0.44,0.53,0.60,0.53,0.47,0.51,0.45,0.47,0.51,0.64,0.45,0.51,0.47,0.44,0.46,0.51,0.46,0.50,0.46,0.51,0.45,0.39,0.52,0.45,0.48,0.50,0.50,0.44,0.59,0.53,0.52,0.53,0.45,0.53,0.54,0.53,0.46,0.40,0.56,0.54,0.55,0.52,0.48,0.53,0.49,0.47,0.49,0.47,0.52,0.51,0.51,0.53,0.59,0.41
NN,0.47,0.55,0.49,0.51,0.52,0.53,0.50,0.46,0.45,0.56,0.52,0.45,0.52,0.54,0.53,0.48,0.53,0.57,0.58,0.50,0.59,0.52,0.49,0.50,0.50,0.55,0.49,0.49,0.49,0.52,0.44,0.57,0.59,0.49,0.43,0.54,0.50,0.47,0.48,0.48,0.55,0.46,0.50,0.45,0.44,0.52,0.51,0.52,0.48,0.51,0.50,0.51,0.56,0.48,0.47,0.51


In [ ]:
df.to_csv('accuracy.csv') 